In [545]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.linear_model import LinearRegression
from scipy.special import rel_entr
import os
# from python_src.compositions import replace_zero_aitchison
# from python_src.compositions import replace_zeroes
from python_src.compositions import constant_aitchison
from python_src.compositions import add_constant
from scipy.spatial.distance import braycurtis
from dataclasses import dataclass

# Global Variables

In [546]:
# Change this depending on your data.
# taxonomic_rank = 'Species'
base_path = "pipelines/camisimGI/bio4"
rel_abund_file = "s1_genus_relabund.csv"
program_name = "Biobakery4"
sample_id = "s1"

expected_input = "pipelines/camisimGI/s1_expected.csv"

pdf_output = PdfPages("figures.pdf")

## Utils

In [547]:
# Use this once scipy fixes the bug.
# from skbio.stats.composition import clr
# from scipy.spatial.distance import euclidean

# def aitchinson_distance(x, y):
#     return euclidean(clr(x), clr(y))

## Colors For Seaborn and MatplotLib

In [548]:
cb_palette = sns.color_palette(as_cmap=True)

# cb_palette = [
# "#e31a1c",
# "#1f78b4",
# "#b2df8a",
# "#a6cee3",
# "#fb9a99",
# "#33a02c",
# ]

print(cb_palette)

color_palette = {
    "Expected": cb_palette[0], 
    "expected": cb_palette[0], 
    "woltka": cb_palette[1], 
    "wol": cb_palette[1], 
    "jams": cb_palette[2], 
    "wgsa": cb_palette[3], 
    "wgsa2": cb_palette[3], 
    "biobakery3": cb_palette[4], 
    "bio3": cb_palette[4], 
    "biobakery4": cb_palette[5], 
    "bio4": cb_palette[5]
}

['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


# One Sample, One Expected

In [549]:
# Generate the expected result from the tsv file. Returns a dataframe.
def generate_expected(input_path: str, plot: bool = False, rank : str = "Genus") -> pd.DataFrame:
    expected = pd.read_csv(input_path, sep=',', index_col=0, names=['Organism', 'Counts', 'TAX_ID'], header=0)
    expected = expected[['Counts']].astype(int)

    # Calculate expected relative abundance. If it's already in RA, this won't change anything.
    expected['RA'] = expected['Counts'] / expected['Counts'].sum()

    if rank == "Species":
        expected.sort_values(by=['RA'], ascending=False, inplace=True)
        return expected

    # Let's split the organism index into two columns to find the genera.
    orgs = expected.index.to_list()
    genus = [org.strip().split(' ')[0] for org in orgs]
    genus = [x.replace('M.', 'Micromonospora') for x in genus]

    # Apparently, propionibacterium have been renamed to cutibacterium.
    genus = [x.replace('Propionibact.', 'Cutibacterium') for x in genus]

    # Add the columns to the dataframe.
    expected['Genus'] = genus
    # display(expected.head(12))

    # Group by genus and sum the counts for overlapping genera.
    exp_genus = expected.groupby('Genus').sum()
    exp_genus.sort_values('RA', ascending=False, inplace=True)

    if plot:
        exp_genus.plot.bar(y='RA', figsize=(8, 5), legend=False, title='Expected Relative Abundance')
    
    return exp_genus

# Use generate_expected to generate the expected result for the bmock12 data.
# exp_genus = generate_expected('pipelines/bmock12/s1_expected_species.csv', False, "Species")
# exp_genus["RA"].to_csv('pipelines/bmock12/s1_expected_species.csv', index_label="Species")

In [550]:
# For camisim, we can just read the csv in directly.
# exp_genus = pd.read_csv(expected_input, index_col=0, names=['Genus', 'RA'], header=0)
# exp_genus = exp_genus.where(exp_genus['RA'] > 0.001).dropna()

# (exp_genus.where(exp_genus['RA'] > 0.001).dropna()).to_csv('pipelines/camisimGI/s2_genus_pretty.csv', index_label="Genus")

In [551]:
def generate_experimental_df(input_path: str, index_name: str) -> pd.DataFrame:
    # Now, load in the experimental values.
    r_genus = pd.read_csv(input_path, index_col=0, names=[index_name, "RA", "TAX_ID"], header=0)
    # display(r_genus.head(12))
    r_genus = r_genus.astype({'RA': 'float64', 'TAX_ID': 'int64'})

    return r_genus

# Instead, let's concat the two dataframes into long format and add a column from where it originated.
def long_format(df1, df2):
    merged = pd.concat([df1, df2], axis=0)

    # !!! This is slick
    merged['Source'] = ['Expected'] * len(df1) + ['Observed'] * len(df2)

    return merged

# result_genus = generate_experimental_df(os.path.join(base_path, rel_abund_file), taxonomic_rank)
# display(result_genus.head(12))
# merged_lf = long_format(exp_genus, result_genus)

# Plotting Tools

In [552]:
def linear_regression(df: pd.DataFrame, exp_df: pd.DataFrame, plot: bool = False, save_path: str = None) -> pd.DataFrame:
    # Merge on the genus key for easy plotting. The expected results are on the left, the observed on the right.
    # linear_df = exp_genus.merge(df, left_index=True, right_index=True)

    # try join because the left merge will drop the genera that are not in the observerd results, but we want to show that the experimental missed it.
    linear_df = exp_df.join(df, how='left', lsuffix='_x', rsuffix='_y')
    linear_df.fillna(0, inplace=True)

    # Linear regression with scikit.
    X = linear_df['RA_x'].values.reshape(-1, 1)
    Y = linear_df['RA_y'].values.reshape(-1, 1)
    reg = LinearRegression().fit(X, Y)
    y_pred = reg.predict(X)

    # Scatter plot of RA_x vs. RA_y.
    if plot:
        fig = plt.figure(figsize=(10, 8))
        plt.scatter(linear_df['RA_x'], linear_df['RA_y'], color='black')

        # Calculate mean absolute error.
        mae = np.mean(np.abs(linear_df['RA_x'] - linear_df['RA_y']))

        # Regression line.
        plt.plot(X, y_pred, color='red', linewidth=2)
        # Labels.
        plt.xlabel('Expected Relative Abundance')
        plt.ylabel('Observed Relative Abundance')
        plt.title(f'Expected vs. Observed Relative Abundance for {program_name} {taxonomic_rank}')

        # Add r^2 value.
        plt.text(0.1, 0.9, f'r^2 = {reg.score(X,Y):.4f}', transform=plt.gca().transAxes)
        # Add MAE.
        plt.text(0.1, 0.85, f'MAE = {mae:.4f}', transform=plt.gca().transAxes)

        # Add line y = x.
        plt.plot([0, 1], [0, 1], color='blue', linewidth=2, linestyle='--')

        if save_path is not None:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        # plt.show()

    return linear_df
    
# linear_regression(result_genus, exp_genus, plot=True, save_path=os.path.join(base_path, f"{sample_id}_bivariate_{taxonomic_rank}.png"))

In [553]:
def fix_x_labels(ax, df, rank):
    xticks = ax.get_xticklabels()
    new_labels = []
    for x in xticks:
        res = df.loc[int(x.get_text()), rank]
        # Get only the first row from the series.
        # This is necessary because if it is unique, it will return a string, but if it is not unique, it will return a series.
        if isinstance(res, pd.Series):
            res = res.iloc[0]
        new_labels.append(res)

    return new_labels

In [554]:
# Bar plot of RA_x vs. RA_y side by side.
# sns.set_style("whitegrid")
def bar_plot(df: pd.DataFrame, plot: bool = False, save_path: str = None, program=program_name, title=f'INSERT TITLE', hue_category='Source', taxonomic_rank = None):
    print("bar plotting")
    df.to_csv("debug.csv", index=True)
    subset = pd.DataFrame()

    # Used to subset by taxonomic rank, but now TAX_ID?
    # Need to test further.
    for x, y in df.groupby("TAX_ID"):
        if y["Source"].values[0] == "Expected":
            subset = pd.concat([y, subset], axis=0)

    display(subset.head())
    subset = subset.where(subset['RA'] > 5e-6).dropna()
    subset.to_csv("subset.csv", index=True)

    print("plotting")
    if plot:
        # Plot a category bar chart with the colors based on the source.
        fig = plt.figure(figsize=(10, 8))
        ax = sns.barplot(x=subset.index, y='RA', hue=hue_category, data=subset, errorbar=None, palette=color_palette)
        ax.semilogy()

        ax.bar_label(ax.containers[0], fmt='%.2e', label_type='center')
        # ax.bar_label(ax.containers[1], fmt='%.2e', label_type='edge')

        # Change the x axis labels from the index to the taxonomic rank.
        ax.set_xticklabels(fix_x_labels(ax=ax, df=subset, rank=taxonomic_rank), rotation=45, horizontalalignment='right')

        # Later, we can fix the y labels.
        # fix_y_labels(ax)

        ax.set_title(title)
        ax.set_xlabel(taxonomic_rank)
        ax.set_ylabel('Relative Abundance')

        if save_path is not None:
            pdf_output.savefig(fig, dpi=300, bbox_inches='tight')
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()

        plt.show()

# bar_plot(merged_lf, plot=True, save_path=os.path.join(base_path, f"{sample_id}_bars_{taxonomic_rank}.png"))

# Multisample
## Utilities

In [555]:
# We will now aggregate the differences by pipeline. 
# To do this, we will start at the root and walk down, looking for "relabund" files and "expected" files.

# root_dir = "pipelines/camisimGI/"

def get_all_expected(root_dir: str, rank="Genus"):
    combined_expected = pd.DataFrame()
    for root, dirs, files in os.walk(root_dir):
        for f in files:
            # print("files: ", files)
            if f"expected_{rank.lower()}_annotated" in f and f.endswith(".csv"):
                print(f)
                df = pd.read_csv(os.path.join(root, f), index_col=0, names=[rank, 'RA', "TAX_ID"], header=0)
                df["Source"] = "Expected"

                # Files are of s#_expected.csv, so we can split on the underscore and take the first part.
                df["SampleID"] = f.split("_")[0]
                combined_expected = pd.concat([combined_expected, df], axis=0)

    return combined_expected

In [556]:
def get_relabund_files(root_dir: str, rank="genus"):
    combined_df = pd.DataFrame()
    for root, dirs, files in os.walk(root_dir):
        for f in files:
            if f"{rank.lower()}_relabund_annotated" in f and f.endswith(".csv"):
                # print(root, f)
                p = os.path.join(root, f)
                exp = generate_experimental_df(p, rank)

                # Add a column to the experimental dataframe with the pipeline name.
                exp['Source'] = os.path.dirname(p).split('/')[-1]

                # Add sampleID to the experimental dataframe.
                exp['SampleID'] = os.path.basename(p).split('_')[0]
                # display(exp.head(10))

                # Add the experimental dataframe to the combined dataframe.
                combined_df = pd.concat([combined_df, exp], axis=0)

    # Ensure that the RA column is a float.
    combined_df['RA'] = combined_df['RA'].astype(float)

    return combined_df

In [557]:
def fully_combined(root_dir, rank) -> pd.DataFrame:
    """
    Gathers all expected and experimental dataframes and combines them into a single dataframe.
    """
    if rank is None:
        raise Exception("Rank is not defined.")

    combined_df = get_relabund_files(root_dir, rank=rank)

    combined_expected = get_all_expected(root_dir, rank=rank)

    # Merge the expected and experimental dataframes.
    merged = pd.concat([combined_expected, combined_df], axis=0)
    merged = merged.reset_index()
    merged = merged.rename(columns={'index': rank})
    merged = merged.set_index("TAX_ID")

    # display(merged.head())
    merged.to_csv("combined.csv", index=True)

    return merged

## Multisample with Different Expected Values

In [558]:
def plot_by_sample(root, output_dir, taxonomic_rank, hue_category='Source'): 
    print("pbs")
    full_df = fully_combined(root, rank=taxonomic_rank)
    
    print("got past full df")
    experiment_name = os.path.basename(root)
    print(root, experiment_name)

    for sample_id, df in full_df.groupby('SampleID'):
        title = f"Expected vs. Observed Relative Abundance for {sample_id} in Experiment {experiment_name} ({taxonomic_rank})"
        display(df.head())
        if output_dir is not None:
            bar_plot(df, plot=True, save_path=os.path.join(output_dir, f"{sample_id}_bars_{taxonomic_rank}_all.png"), title=title, taxonomic_rank=taxonomic_rank, hue_category=hue_category)
        else:
            bar_plot(df, plot=True, save_path=None, title=title, taxonomic_rank=taxonomic_rank, hue_category=hue_category)

# plot_by_sample(root_dir, hue_category='SampleID')

In [559]:
def linear_regression(X, Y):
    reg = LinearRegression().fit(X, Y)
    y_pred = reg.predict(X)

    return reg, y_pred

def linear_plot(input_df: pd.DataFrame, title, sample_id, hue_category="Source_observed", save_path=None, inset=False, colors=color_palette) -> pd.DataFrame:
    """
    Plot a linear regression of the expected vs. observed relative abundance. Also calculates the R^2 value, MAE, and Aitchison distance.
    Parameters:
        input_df: A dataframe with the expected and observed relative abundance. The third column should be the source of the data.
        title: The title of the plot.
        sample_id: The sample ID of the plot.
        hue_category: The category to use for the hue (default: "Source_observed").
        save_path: The path to save the plot to (if None, the plot will not be saved).
        inset: Whether or not to plot the inset.
    Returns:
        A dataframe with the R^2, MAE, and Aitchison distance with pipeline and sampleID.
    """
    input_df.to_csv("bivariate.csv", index=True)
    stats_df = pd.DataFrame()

    fig = plt.figure(figsize=(10, 10))

    ax = sns.lmplot(x="RA_expected", y="RA_observed", hue=hue_category, data=input_df, fit_reg=True, height=7, aspect=11/7, ci=None, palette=colors)

    # Make linear regression for each pipeline (or sampleID for replicates).
    pipeline_offset = -0.1
    for heading, dataframe in input_df.groupby(hue_category):
        old_x = dataframe["RA_expected"].values.reshape(-1, 1)
        old_y = dataframe["RA_observed"].values.reshape(-1, 1)

        # x, y = replace_zeroes(old_x), replace_zeroes(old_y)
        x, y = add_constant(old_x), add_constant(old_y)

        reg, y_pred = linear_regression(x, y)

        # Calculate R^2.
        r2 = reg.score(x, y)

        # Add r^2
        plt.text(0.1, pipeline_offset, f'r\u00b2 = {r2:.4f} for {heading}', transform=plt.gca().transAxes)

        # Calculate MAE.
        mae = np.mean(np.abs(x - y))

        # Add MAE
        plt.text(0.4, pipeline_offset, f'MAE = {mae:.4f} for {heading}', transform=plt.gca().transAxes)

        # Add the aitchison distance.
        try: 
            # a_d = replace_zero_aitchison(dataframe['RA_expected'].values, dataframe['RA_observed'].values)
            a_d = constant_aitchison(dataframe['RA_expected'].values, dataframe['RA_observed'].values)
            plt.text(0.7, pipeline_offset, f'Aitchison = {a_d:.4f} for {heading}', transform=plt.gca().transAxes)

            # Make a dataframe with sampleID as the index and the R^2, MAE, and Aitchison distance as the columns.
            stats = {'R^2': r2, 'MAE': mae, 'Aitchison': a_d}
            stats_df = pd.concat([stats_df, pd.DataFrame(stats, index=[heading])])


        except ValueError as e:
            print(f"ValueError: {e}")
            # If there is a zero in the data, we cannot calculate the aitchison distance.
            plt.text(0.7, pipeline_offset, f'Aitchison = N/A for {heading}', transform=plt.gca().transAxes)

            # s = pd.Series({'SampleID': sample_id, 'Pipeline': heading, 'R^2': r2, 'MAE': mae, 'Aitchison': np.nan}) 
            # stats_df = pd.concat([stats_df, s], ignore_index=True)

        """
        # Add the bray-curtis distance.
        try: 
            a_d = 1 - braycurtis(dataframe['RA_expected'].values, dataframe['RA_observed'].values)
            plt.text(0.7, pipeline_offset, f'1-BC = {a_d:.4f} for {heading}', transform=plt.gca().transAxes)
        except ValueError:
            plt.text(0.7, pipeline_offset, f'1-BC = N/A for {heading}', transform=plt.gca().transAxes)
        """

        # Calculate relative entropy using scipy.special.rel_entr
        # Not sure if this works since sometimes the observed is smaller than the expected.
        # re = rel_entr(dataframe['RA_expected'], dataframe['RA_observed']).sum()
        # print(re)
        # Add relative entropy
        # plt.text(0.7, pipeline_offset, f'Relative Entropy = {re:.4f} for {heading}', transform=plt.gca().transAxes)

        # Move the offset down. 
        pipeline_offset -= 0.05

    # Add title.
    plt.title(title)

    # Add y = x line.
    ## Get the max x and max y values.
    max_x = input_df['RA_expected'].max()
    max_y = input_df['RA_observed'].max()
    max_val = max(max_x, max_y)

    # Plot a line from (0, 0) to (max_x+0.1, max_y+0.1)
    plt.plot([0, max_val + 0.01], [0, max_val + 0.01], ls="--", c=".3")

    # Add an inset for the x values between 0 and 0.05.
    if inset:
        left, bottom, width, height = [0.65, 0.15, 0.25, 0.25]
        ax2 = ax.fig.add_axes([left, bottom, width, height])
        ax2 = sns.scatterplot(x="RA_expected", y="RA_observed", hue=hue_category, data=input_df, ax=ax2, legend=False, palette=colors)
        ax2.set_xlim(-0.001, 0.02)
        ax2.set_ylim(-0.001, 0.02)
        ax2.set_title("Zoomed In")

    if save_path is not None:
        pdf_output.savefig(ax.figure, bbox_inches='tight', dpi=300)
        plt.close(ax.figure)
        

    return stats_df

In [560]:
def linear_plot_log(input_df: pd.DataFrame, title, sample_id, hue_category="Source_observed", save_path=None):
    # We want to plot the x and y axis on a log scale.
    fig = plt.figure(figsize=(20, 20))
    ax = sns.lmplot(x="RA_expected", y="RA_observed", hue=hue_category, data=input_df, fit_reg=True, height=7, aspect=11/7, ci=None, truncate=True)
    # ax.set(xscale="symlog", yscale="symlog")

In [561]:
# Since the sampleID and pipeline are changed depending on the hue color (for replicates), we need to change it here.
def cleanup_bivariate_stats(df: pd.DataFrame, sampleID: str):
    # We need to add a column equal to sampleID.
    df['SampleID'] = sampleID

    # Pop the old index.
    df.reset_index(inplace=True)

    # Set the index as the sampleID.
    df.set_index(['SampleID'], inplace=True)

    # We need to rename the "index" column to pipeline.
    df.rename(columns={'index': 'Pipeline'}, inplace=True)

    return df

In [562]:

# Convert from Genus, RA, Source, SampleID to Genus RA_x, RA_y, Source, SampleID.
def convert_to_bivariate(root_dir: str, save_path=None, inset=False, rank=None) -> pd.DataFrame:
    if rank is None:
        raise Exception("Rank cannot be None.")

    df = fully_combined(root_dir, rank=rank)
    combined_stats = pd.DataFrame()
    for sample, sample_df in df.groupby('SampleID'):
        bivariate_df = pd.DataFrame()
        # Get the expected dataframe.
        expected = sample_df[sample_df['Source'] == 'Expected']
        # display(expected)

        # Get the dirname of the root directory.
        dirname = root_dir.split('/')[-1]
        # print(dirname)

        # Get the experimental dataframe.
        experimental = sample_df[(sample_df['Source'] != 'Expected') & (sample_df['Source'] != dirname)]
        # display(experimental)

        # Merge the expected and experimental dataframes.
        merged = pd.merge(expected, experimental, on="TAX_ID", how='left', suffixes=('_expected', '_observed'))
        merged['SampleID'] = sample
        # merged["RA_observed"] = merged["RA_observed"].fillna(0)

        # Add the merged dataframe to the bivariate dataframe.
        bivariate_df = pd.concat([bivariate_df, merged], axis=0)

        title = f"Bivariate Linear Regression for Sample {sample} in Experiment {dirname}"

        # bivariate_df = bivariate_df.sort_values(by='RA_expected', ascending=False).head(30)
        if save_path is not None:
            stats_df = linear_plot(bivariate_df, title, sample, hue_category="Source_observed", save_path=os.path.join(root_dir, f"{sample}_linear_{rank}_all.png"), inset=inset)
            stats_df = cleanup_bivariate_stats(stats_df, sample)

            combined_stats = pd.concat([combined_stats, stats_df], axis=0)
        else:
            stats_df = linear_plot(bivariate_df, title, sample, hue_category="Source_observed", save_path=None)
            stats_df = cleanup_bivariate_stats(stats_df, sample)

            combined_stats = pd.concat([combined_stats, stats_df], axis=0)

    return combined_stats
# bivariate_df = convert_to_bivariate(fully_combined())

## Multiple Samples (Replicates) with One Expected

In [563]:
# This function is used for multiple samples from the same pipeline against the same expected.
def plot_many_versus_expected(root_dir, output_dir, in_df, rank):
    exp = get_all_expected(root_dir, rank)
    dirname = root_dir.split('/')[-1]
    for pipeline, df in in_df.groupby('Source'):
        print(pipeline)
        if pipeline == 'Expected':
            continue
        
        if pipeline != dirname:
            fig = plt.figure(figsize=(15, 12))

            df = df.where(df['RA'] > 0.001).dropna()

            # Add the expected dataframe to the combined dataframe.
            merged = pd.concat([exp, df], axis=0)

            # Sample names change, so we can just use the same set of colors rather than having to change them.
            ax = sns.barplot(x=merged.index, y='RA', hue="SampleID", data=merged, errorbar=None, log=True, palette=cb_palette)
            ticks = [0.001, 0.01, 0.10]
            ax.set_yticks(ticks)
            ax.set_yticklabels(ticks)
            # ax.bar_label(ax.containers[0], fmt='%.2e', label_type='center')
            # ax.bar_label(ax.containers[1], fmt='%.2e', label_type='edge')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

            title = f"Expected vs. Observed Relative Abundance for {rank} using {pipeline} in Experiment {dirname}"
            ax.set_title(title)
            ax.set_xlabel('Genus')
            ax.set_ylabel('Relative Abundance')

            pdf_output.savefig(fig, bbox_inches='tight', dpi=300)
            plt.close(fig)
            # plt.savefig(os.path.join(output_dir, f"{pipeline}", f"{pipeline}_bars.png"), dpi=300, bbox_inches='tight')
        else:
            continue

# plot_many_versus_expected(root_dir, fully_combined(), "genus")

In [564]:
# This function will plot the expected vs. observed for each sample in each pipeline.
# There is only one expected value for each sample since they are replicates.
# This will be a bivariate plot.
def plot_many_versus_expected_bivariate(root_dir, observed_df, expected_df, rank, inset) -> pd.DataFrame:
    """
    This function will plot the expected vs. observed for each sample in each pipeline. 
    There is only one expected value for each sample since they are replicates. 
    This will be a bivariate plot that will include MAE, r^2 and Aitchison distance.

    Parameters
    ----------
    root_dir : str
        The root directory of the pipeline.
    observed_df : pd.DataFrame
        The observed dataframe.
    expected_df : pd.DataFrame
        The expected dataframe.
    rank : str
        The taxonomic rank.
    inset : bool
        Whether or not to include an inset plot.

    Returns
    -------
    pd.DataFrame
        A dataframe containing the statistics for each sample.
    """
    # First, we need to make a df of observed vs. expected for each sample.
    combined_stats = pd.DataFrame()
    for pipeline, pipeline_df in observed_df.groupby("Source"):
        # Plotting expected vs expected is useless.
        if pipeline == "tourlousse" or pipeline == "Expected":
            continue

        merged = pd.merge(expected_df.copy(), pipeline_df, on='Genus', how='left', suffixes=('_expected', '_observed'))

        save_path = os.path.join(root_dir, pipeline, f"{pipeline}_bivariate_{rank}_all_samples.png")
        # merged.to_csv(os.path.join(root_dir, pipeline, f"{pipeline}_bivariate_{rank}_all_samples.csv"))

        exp_name = os.path.basename(root_dir)

        # Add amos to the mixed or hilo samples since they are nested.
        if exp_name == "mixed" or exp_name == "hilo":
            exp_name = "Amos " + exp_name

        pipeline_stats = linear_plot(merged, f"Expected vs. Observed Relative Abundance for {rank} using {pipeline} in Experiment {exp_name}", pipeline, hue_category="SampleID_observed", save_path=save_path, inset=inset, colors=cb_palette)

        # Add pipeline column to the stats dataframe equal to the pipeline name.
        pipeline_stats['Pipeline'] = pipeline

        # Add the pipeline stats to the combined stats dataframe.
        combined_stats = pd.concat([combined_stats, pipeline_stats], axis=0)

    return combined_stats

# plot_many_versus_expected_bivariate(get_relabund_files(root_dir), get_all_expected(root_dir, "genus"), "genus")

# Run the code here: MAIN.

In [565]:
# We can initialize the dataclasses to hold the parameters for each pipeline.
@dataclass
class Pipeline:
    root: str
    inset: bool

    def __init__(self, root: str, inset: bool):
        self.root = root
        self.inset = inset

bmock12 = Pipeline("pipelines/bmock12", False)
camisim = Pipeline("pipelines/camisimGI", True)
tourlousse = Pipeline("pipelines/tourlousse", False)
amos_hilo = Pipeline("pipelines/amos/hilo", False)
amos_mixed = Pipeline("pipelines/amos/mixed", False)

# No ending slash for these directories.
root_dirs_one_to_one = [bmock12, camisim]
# root_dirs_one_to_many = [tourlousse, amos_hilo, amos_mixed]
root_dirs_one_to_many = [tourlousse]

In [566]:
def main():
    def one_to_one():
        for exp in root_dirs_one_to_one:
            print("Plotting bars: ")
            plot_by_sample(root=exp.root, output_dir=exp.root, hue_category="Source", taxonomic_rank="Genus")
            plot_by_sample(root=exp.root, output_dir=exp.root, hue_category="Source", taxonomic_rank="Species")

            print("Plotting bivariate: ")
            final_stats = convert_to_bivariate(exp.root, save_path=exp.root, inset=exp.inset, rank="Genus")
            final_stats = convert_to_bivariate(exp.root, save_path=exp.root, inset=exp.inset, rank="Species")
            final_stats.to_csv(os.path.join(exp.root, "all_stats_replicates.csv"), index_label="SampleID")

    def many_to_one():
        for exp in root_dirs_one_to_many:
            plot_many_versus_expected(exp.root, exp.root, fully_combined(exp.root), "genus")
            final_stats = plot_many_versus_expected_bivariate(exp.root, fully_combined(exp.root), get_all_expected(exp.root, "genus"), "genus", inset=exp.inset)
            final_stats.to_csv(os.path.join(exp.root, "all_stats_replicates.csv"), index_label="SampleID")

    many_to_one()
    pdf_output.close()
    
main()

# fc = fully_combined("pipelines/bmock12/")
# display(fc)

pbs
S2_expected_genus_annotated.csv
S1_expected_genus_annotated.csv
got past full df
pipelines/camisimGI camisimGI


,Genus,RA,Source,SampleID
TAX_ID,,,,
517,Bordetella,0.561947,Expected,S1
1485,Clostridium,0.225664,Expected,S1
816,Bacteroides,0.123894,Expected,S1
1784836,Fermentimonas,0.013274,Expected,S1
1508657,Ruminiclostridium,0.008850,Expected,S1


bar plotting


,Genus,RA,Source,SampleID
TAX_ID,,,,
2039240,Anaerotignum,0.000000,Expected,S1
1870884,Clostridioides,0.000000,Expected,S1
1870884,CLOSTRIDIOIDES,0.000003,jams,S1
1870884,Clostridioides,0.000000,wgsa,S1
1822464,Paraburkholderia,0.000000,Expected,S1


plotting


...

plotting
doing genus stats
S2_expected_genus_annotated.csv
S1_expected_genus_annotated.csv
doing species stats
S1_expected_species_annotated.csv
S2_expected_species_annotated.csv


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>